# 리소스 예측을 통한 전략적 운용
효과적인 리소스 관리와 예측은 프로젝트의 성공적인 완료와 직결됩니다. 리소스 예측을 통해 프로젝트에서 요구되는 인적 자원 및 시간 관리를 더욱 투명하고 효율적으로 수행할 수 있습니다. 이는 프로젝트의 각 단계에서 필요한 리소스를 정확히 예측하고, 적절한 시기에 배치할 수 있도록 도와줍니다. 이러한 예측 능력은 프로젝트 운영의 투명성을 높이고, 불필요한 낭비를 줄이며, 전체적인 생산성을 향상시키는 데 중요한 역할을 합니다.

## 1 데이터 불러들이기
먼저, 필요한 라이브러리를 불러오고 resource_event_month.csv 파일을 읽어들여 데이터프레임으로 저장합니다. 데이터를 확인하기 위해 첫 몇 행을 출력합니다.

In [ ]:
# [TAKS] 1 데이터 불러들이기


## 2 시각화해서 확인하기
Plotly를 사용하여 이벤트와 월별로 리소스 사용량을 시각화합니다. 각 이벤트를 순서대로 나열하고, 이벤트와 월을 결합한 새로운 컬럼 event_month를 생성합니다. 그런 다음, 역할과 프로젝트 클래스에 따라 여러 개의 차트를 생성하여 MM(Man Month) 트렌드를 시각화합니다.

In [ ]:
# 2 시각화해서 확인하기
import plotly.express as px

events = ['M0', 'P0', 'P1', 'M', 'SOP']
resource_event_month['event'] = pd.Categorical(resource_event_month['event'], categories=events, ordered=True)
resource_event_month['event_month'] = resource_event_month['event'].astype(str) + resource_event_month['month'].astype(str)
event_month_key = (resource_event_month[['event', 'month']]
                   .drop_duplicates()
                   .sort_values(['event', 'month'])
                   .assign(event_month=lambda x: x['event'].astype(str) + "+" + x['month'].astype(str)))

fig = px.line(resource_event_month, x='event_month', y='mm', 
              facet_row='role', facet_col='project_class', 
              markers=True, title='MM Trend by Event and Month',
              labels={'event_month': 'Event + Month', 'mm': 'Man Month'},
              category_orders={'event_month': event_month_key})
fig.update_xaxes(tickangle=45)
fig.update_layout(height=800, width=1200)
fig.show()

## 3 선형 회귀를 통한 리소스 예측
선형 회귀 분석을 통해 리소스 사용량을 예측합니다. event, month, project_class를 독립 변수로 사용하여 MM(Man Month)을 종속 변수로 하는 회귀 모델을 구축합니다. 모델의 예측값을 계산하고, 원본 데이터프레임에 추가합니다.

In [ ]:
# 3 선형 회귀를 통한 리소스 예측
from sklearn.linear_model import LinearRegression

X = resource_event_month[['event', 'month', 'project_class']]
X = pd.get_dummies(X, drop_first=True)  # Convert categorical variables to dummy/indicator variables
y = resource_event_month['mm']

model = LinearRegression()
model.fit(X, y)

predictions = model.predict(X)
resource_event_month['pred'] = predictions
resource_event_month

> 질문: 논리적으로 볼 때, 위의 회귀 모델에서 어떤 점이 잘못되었고, 어떻게 수정하면 좋을까요?